In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
VWAGY_path = Path("../files/VWAGY.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
vwagy_df = pd.read_csv(VWAGY_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([vwagy_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,26.635000,26.730000,26.490000,26.730000,20.591963,24800.0,13.41,188.309998,0.09,25.822500
2014-05-01,26.683500,26.683500,26.590000,26.594999,20.487959,13800.0,13.25,188.330002,0.09,26.000000
2014-05-02,26.125000,26.230000,26.059999,26.120001,20.122036,38600.0,12.91,188.059998,0.09,25.450001
2014-05-05,25.785000,26.294001,25.570000,26.264999,20.233738,429000.0,13.29,188.419998,0.09,25.455000
2014-05-06,25.879999,26.035000,25.760000,25.975000,20.010332,100200.0,13.80,186.779999,0.09,25.067499
2014-05-07,25.915001,26.065001,25.757500,25.822500,19.892851,75800.0,13.40,187.880005,0.09,25.398500
2014-05-08,26.105000,26.245001,25.969999,26.000000,20.029591,93000.0,13.43,187.679993,0.09,25.375000
2014-05-09,25.940001,25.945000,25.264999,25.450001,19.605890,101200.0,12.92,187.960007,0.09,25.600000
2014-05-12,25.945000,25.995001,25.400000,25.455000,19.609743,42800.0,12.23,189.789993,0.09,25.717501
2014-05-13,25.225000,25.379999,25.010000,25.067499,19.311224,41000.0,12.13,189.960007,0.09,25.684999


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,14.69,14.790,14.65,14.725,14.725,193000.0,14.33,512.859985,5.33,15.040,14.93425,51.509069,15.796342,14.93425,14.072158
2024-03-19,14.87,14.980,14.84,14.920,14.920,222100.0,13.82,515.710022,5.33,15.144,14.95925,54.395795,15.786861,14.95925,14.131639
2024-03-20,14.79,15.088,14.76,15.050,15.050,150700.0,13.04,520.479980,5.33,15.200,14.97425,56.265044,15.796751,14.97425,14.151749
2024-03-21,14.97,15.010,14.78,14.850,14.850,180700.0,12.92,522.200012,5.33,15.250,14.96225,52.687066,15.784644,14.96225,14.139856
2024-03-22,14.78,14.840,14.73,14.830,14.830,103500.0,13.06,521.210022,5.33,15.200,14.94125,52.328703,15.754100,14.94125,14.128400


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,24800.0,13.41,188.309998,0.09,25.822500,NaN,NaN,NaN,NaN,NaN
2014-05-01,13800.0,13.25,188.330002,0.09,26.000000,NaN,0.000000,NaN,NaN,NaN
2014-05-02,38600.0,12.91,188.059998,0.09,25.450001,NaN,0.000000,NaN,NaN,NaN
2014-05-05,429000.0,13.29,188.419998,0.09,25.455000,NaN,20.641122,NaN,NaN,NaN
2014-05-06,100200.0,13.80,186.779999,0.09,25.067499,NaN,14.288641,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/VWAGY_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,107800.0,11.68,191.380005,0.09,25.915001,25.88305,54.503645,26.742640,25.88305,25.023460
2014-05-29,56800.0,11.57,192.369995,0.09,26.500000,25.86355,56.915645,26.657424,25.86355,25.069676
2014-05-30,33200.0,11.40,192.679993,0.09,26.559999,25.84680,54.543018,26.588089,25.84680,25.105511
2014-06-02,106000.0,11.58,192.899994,0.10,26.370001,25.86830,60.905223,26.665766,25.86830,25.070834
2014-06-03,292800.0,11.87,192.800003,0.10,26.045000,25.87005,53.901398,26.671326,25.87005,25.068774


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 650us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 387us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 495us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 364us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 501us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 569us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 341us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 566us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 597us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 360us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 550us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 492us/step
Average R-squared (Train): -12.482407673821026
Average R-squared (Test): -12.647223430976645


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 661us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 542us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 622us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 544us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 628us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 562us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 595us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 500us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 584us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step
Model 1 - Train R-squared: 0.8469928759803589, Test R-squared: 0.8443621602858393, Train MAE: 2.1749916985932236, Train MSE: 5.0766989976319135, Test MAE: 2.1973591907801864, Test MSE: 5.161303597781666
Model 2 - Train R-squared: 0.8222760123088653, Test R-squared: 0.8297902172738143, Train MAE: 2.2502302695877767, Train MSE: 5.893619962463865, Test MAE: 2.217377599376112, Test MSE: 5.67768384413742
Model 3 - Train R-squared: 0.8591227740724234, Test R-squared: 0.8388157626404102, Train MAE: 2.0584490078282465, Train MSE: 4.738166484836037, Test MAE: 2.0434443562815447, Test MSE: 4.675981158373182
Model 4 - Train R-squared: 0.8228867075047647, Test R-squared: 0.8144050634177395, Train MAE: 2.1666265893275423, Train MSE: 5.860009417757145, Test MAE: 2.191311389688322, Test MSE: 6.321287896842886
Model 5 - Train R-squared: 0.8755055207812587, Test R-squared: 0.8894434985529827, Train MAE: 1.8646036166846836, 

In [20]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.8469928759803589, Test R-squared: 0.8443621602858393, Train MAE: 2.1749916985932236, Train MSE: 5.0766989976319135, Test MAE: 2.1973591907801864, Test MSE: 5.161303597781666


In [21]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step


,Actual,Predicted
date,,
2014-05-28,26.235001,20.604481
2014-05-29,26.340000,20.811102
2014-05-30,26.260000,20.816929
2014-06-02,26.549999,20.819036
2014-06-03,26.299999,20.549768
...,...,...
2024-03-18,14.725000,12.954198
2024-03-19,14.920000,13.054773
2024-03-20,15.050000,13.159608


In [22]:
predictions_df.to_csv('../predicted_data/VWAGY_predicted_data.csv', index=True)

In [23]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
               Actual  Predicted
date                            
2014-05-28  26.235001  20.604481
2014-05-29  26.340000  20.811102
2014-05-30  26.260000  20.816929
2014-06-02  26.549999  20.819036
2014-06-03  26.299999  20.549768
...               ...        ...
2024-03-18  14.725000  12.954198
2024-03-19  14.920000  13.054773
2024-03-20  15.050000  13.159608
2024-03-21  14.850000  13.117736
2024-03-22  14.830000  13.107313

[2473 rows x 2 columns]


In [24]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,26.235001,20.604481,-21.461864
2014-05-29,26.340000,20.811102,-20.990501
2014-05-30,26.260000,20.816929,-20.727613
2014-06-02,26.549999,20.819036,-21.585547
2014-06-03,26.299999,20.549768,-21.863995
...,...,...,...
2024-03-18,14.725000,12.954198,-12.025821
2024-03-19,14.920000,13.054773,-12.501519
2024-03-20,15.050000,13.159608,-12.560745


In [25]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 18.57050534245352
